In [1]:
import os
os.chdir("../")
%pwd

'd:\\AI\\NLP\\HandsOn\\sentiment-analysis'

In [3]:
from dataclasses import dataclass
from pathlib import Path
import numpy as np
from typing import Dict, List, Tuple, Union, Optional, Any

@dataclass(frozen=True)
class ModelDevelopmentConfig:
    root_dir: Path
    data_files_path: Path

    
@dataclass
class TrainingResult:
    """Stores the results of model training and evaluation"""
    model_name: str
    metrics: Dict[str, Any]
    predictions: np.ndarray
    parameters: Dict[str, Any]
    trained_model: Any  # Store the trained model instance


In [4]:
from SentiScope.constants import (CONFIG_FILE_PATH,
                                  PARAMS_FILE_PATH)
from SentiScope.utils.file_utils import (create_directories,
                                            get_size)
from SentiScope.utils.config_utils import (read_yaml,
                                           Settings,
                                           get_settings)

In [5]:
import json
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_latest_report_baselinemodels(self) -> Dict:
        """Locate the latest metadata.json file based on the timestamp folder."""
        config = self.config.feature_transformation
        profiling_dir = Path(config.root_dir)

        # Get all subdirectories in data_profiling
        timestamp_dirs = [d for d in profiling_dir.iterdir() if d.is_dir()]
        
        if not timestamp_dirs:
            raise FileNotFoundError("No timestamp folders found in feature_transformation.")

        # Sort directories by name (assuming timestamp format)
        latest_dir = sorted(timestamp_dirs, key=lambda x: x.name, reverse=True)[0]
        metadata_path = latest_dir / "metadata.json"

        if not metadata_path.exists():
            raise FileNotFoundError(f"metadata.json not found in {latest_dir}.")

        # Load the report.json file
        with open(metadata_path, "r") as f:
            report_data = json.load(f)

        return report_data

    def get_model_development_config(self) -> ModelDevelopmentConfig:
        config = self.config.model_development
        report_data = self.get_latest_report_baselinemodels()

        create_directories([config.root_dir])

        timestamp = report_data["timestamp"]
        # data_file_path = Path(str(config.data_file).format(timestamp=timestamp))
        data_files_path = Path(config.data_files_path).joinpath(f"{timestamp}")
 

        model_development_config = ModelDevelopmentConfig(
            root_dir=config.root_dir,
            data_files_path=data_files_path,

        )

        return model_development_config

In [6]:
from abc import ABC, abstractmethod
from typing import Dict, Any
import numpy as np

class BaseModel(ABC):
    """Abstract base class defining the interface for all models"""
    
    @abstractmethod
    def train(self, X_train: Any, y_train: Any) -> None:
        """Train the model on the given data"""
        pass
    
    @abstractmethod
    def predict(self, X: Any) -> np.ndarray:
        """Make predictions on new data"""
        pass
    
    @abstractmethod
    def get_params(self) -> Dict[str, Any]:
        """Get the model's current parameters"""
        pass
    
    @abstractmethod
    def set_params(self, **params) -> None:
        """Set the model's parameters"""
        pass

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


class SklearnModelWrapper(BaseModel):
    """Wrapper for scikit-learn models to conform to our interface"""
    
    def __init__(self, model):
        self.model = model
    
    def train(self, X_train, y_train):
        return self.model.fit(X_train, y_train)
    
    def predict(self, X):
        return self.model.predict(X)
    
    def get_params(self):
        return self.model.get_params()
    
    def set_params(self, **params):
        return self.model.set_params(**params)

class LogisticRegressionModel(SklearnModelWrapper):
    """Logistic Regression implementation"""
    
    def __init__(self):
        super().__init__(LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000))

class SVMModel(SklearnModelWrapper):
    """Support Vector Machine implementation"""
    
    def __init__(self):
        super().__init__(SVC(kernel='linear', probability=True))

In [11]:
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Any, List
from sklearn.metrics import classification_report
import numpy as np
from scipy.sparse import issparse
from scipy import sparse
import os
import json
from datetime import datetime
from SentiScope.logging import logger
from datetime import datetime

class TrainingManager:
    """Handles model training and evaluation"""
    
    def __init__(self, output_dir: Path):
        self.training_history: List[TrainingResult] = []
        self.output_dir = output_dir
        logger.info(f"Initialized TrainingManager with output directory: {output_dir}")
    
    def _validate_data_split(self, data_split):
        """Validate the data split dictionary"""
        logger.info("Validating data split...")
        
        if not isinstance(data_split, dict):
            msg = f"data_split must be a dictionary, got {type(data_split)}"
            logger.error(msg)
            raise TypeError(msg)
            
        required_keys = ["X_train", "X_test", "y_train", "y_test"]
        
        missing = [key for key in required_keys if key not in data_split]
        if missing:
            msg = f"Missing required keys in data_split: {missing}"
            logger.error(msg)
            raise ValueError(msg)
        
        n_train_samples = (data_split["X_train"].shape[0] if not issparse(data_split["X_train"]) 
                          else data_split["X_train"].shape[0])
        n_test_samples = (data_split["X_test"].shape[0] if not issparse(data_split["X_test"]) 
                         else data_split["X_test"].shape[0])
        
        if n_train_samples != len(data_split["y_train"]):
            msg = (f"Mismatch in training set dimensions: X_train has {n_train_samples} "
                  f"samples but y_train has {len(data_split['y_train'])} samples")
            logger.error(msg)
            raise ValueError(msg)
        
        if n_test_samples != len(data_split["y_test"]):
            msg = (f"Mismatch in test set dimensions: X_test has {n_test_samples} "
                  f"samples but y_test has {len(data_split['y_test'])} samples")
            logger.error(msg)
            raise ValueError(msg)
        
        logger.info("Data split validation completed successfully")
    
    def train_and_evaluate(self, model, model_name: str, data_split) -> TrainingResult:
        """Train a model and evaluate its performance"""
        try:
            logger.info(f"Starting training and evaluation for {model_name}")
            
            # Validate the data split
            self._validate_data_split(data_split)
            
            # Verify model has required methods
            required_methods = ['train', 'predict', 'get_params']
            for method in required_methods:
                if not hasattr(model, method):
                    msg = f"Model lacks required method: {method}"
                    logger.error(msg)
                    raise AttributeError(msg)
            
            # Train the model
            logger.info(f"Training {model_name}...")
            model.train(data_split["X_train"], data_split["y_train"])
            
            # Make predictions
            logger.info(f"Making predictions for {model_name}...")
            predictions = model.predict(data_split["X_test"])
            
            # Calculate metrics
            logger.info(f"Calculating metrics for {model_name}...")
            metrics = classification_report(
                data_split["y_test"], 
                predictions, 
                output_dict=True
            )
            
            # Create result object with trained model
            result = TrainingResult(
                model_name=model_name,
                metrics=metrics,
                predictions=predictions,
                parameters=model.get_params(),
                trained_model=model
            )
            
            # Save results to output directory
            model_dir = self.output_dir / model_name
            model_dir.mkdir(exist_ok=True)
            
            # Save predictions
            np.save(model_dir / 'predictions.npy', predictions)
            
            # Save metrics and parameters
            with open(model_dir / 'results.json', 'w') as f:
                json.dump({
                    'metrics': metrics,
                    'parameters': model.get_params()
                }, f, indent=4)
            
            self.training_history.append(result)
            
            logger.info(f"Successfully completed training and evaluation for {model_name}")
            return result
            
        except Exception as e:
            logger.error(f"Error in train_and_evaluate for {model_name}: {str(e)}")
            raise

In [12]:
import joblib
class SentimentPipeline:
    """Coordinates all components of the sentiment analysis system"""
    
    def __init__(self, config: ModelDevelopmentConfig):
        self.config = config
        self.data_files_path = self.config.data_files_path
        
        # Create timestamped output directory
        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        self.output_dir = Path(self.config.root_dir) / self.timestamp
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
        # Create models directory
        self.models_dir = self.output_dir / 'models'
        self.models_dir.mkdir(exist_ok=True)
        
        self.training_manager = TrainingManager(self.output_dir)
        self.models = {
            'logistic_regression': LogisticRegressionModel()
        }
        
        logger.info(f"Initialized SentimentPipeline with output directory: {self.output_dir}")
    
    def prepare_data(self):
        """Load and prepare data for training"""
        try:
            logger.info("Loading prepared data from artifacts...")
            
            # Construct paths to the data files
            x_train_path = self.data_files_path / "X_train.npy"
            x_test_path = self.data_files_path / "X_test.npy"
            y_train_path = self.data_files_path / "y_train.npy"
            y_test_path = self.data_files_path / "y_test.npy"

            # Load the data
            X_train_npy = np.load(x_train_path, allow_pickle=True)
            X_test_npy = np.load(x_test_path, allow_pickle=True)
            y_train = np.load(y_train_path, allow_pickle=True)
            y_test = np.load(y_test_path, allow_pickle=True)
            
            # Convert to sparse matrices
            X_train_sparse = sparse.csr_matrix(X_train_npy.all())
            X_test_sparse = sparse.csr_matrix(X_test_npy.all())

            logger.info("Data successfully loaded and converted to sparse format")
            
            # Save data info to metadata
            metadata = {
                'timestamp': self.timestamp,
                'data_shapes': {
                    'X_train': X_train_sparse.shape,
                    'X_test': X_test_sparse.shape,
                    'y_train': y_train.shape,
                    'y_test': y_test.shape
                },
                'data_source': {
                    'X_train': str(x_train_path),
                    'X_test': str(x_test_path),
                    'y_train': str(y_train_path),
                    'y_test': str(y_test_path)
                }
            }
            
            with open(self.output_dir / 'metadata.json', 'w') as f:
                json.dump(metadata, f, indent=4)
            
            return X_train_sparse, X_test_sparse, y_train, y_test
            
        except Exception as e:
            logger.error(f"Error loading data: {str(e)}")
            raise

    def save_model(self, model, name: str):
        """Save a trained model to disk"""
        try:
            model_path = self.models_dir / f"{name}.joblib"
            joblib.dump(model, model_path)
            logger.info(f"Saved model {name} to {model_path}")
            return str(model_path)
        except Exception as e:
            logger.error(f"Error saving model {name}: {str(e)}")
            raise
    
    def train_models(self):
        """Train all registered models"""
        try:
            logger.info("Starting model training pipeline")
            
            x_train, x_test, y_train, y_test = self.prepare_data()
            data_split = {
                "X_train": x_train,
                "X_test": x_test,
                "y_train": y_train,
                "y_test": y_test
            }
            
            results = {}
            model_paths = {}
            for name, model in self.models.items():
                logger.info(f"Training model: {name}")
                # Train and evaluate the model
                training_result = self.training_manager.train_and_evaluate(
                    model, name, data_split
                )
                results[name] = training_result
                
                # Save the trained model from the training result
                model_path = self.save_model(training_result.trained_model, name)
                model_paths[name] = model_path
            
            # Save final summary
            summary = {
                'timestamp': self.timestamp,
                'models_trained': list(self.models.keys()),
                'model_paths': model_paths,
                'results': {
                    name: {
                        'metrics': result.metrics,
                        'parameters': result.parameters,
                        'model_path': model_paths[name]
                    }
                    for name, result in results.items()
                }
            }
            
            with open(self.output_dir / 'training_summary.json', 'w') as f:
                json.dump(summary, f, indent=4)
            
            logger.info("Model training pipeline completed successfully")
            return results
            
        except Exception as e:
            logger.error(f"Error in train_models: {str(e)}")
            raise

In [13]:
config = ConfigurationManager()
model_development_config = config.get_model_development_config()
pipeline = SentimentPipeline(config=model_development_config)
results = pipeline.train_models()
print(results)

[2025-01-17 07:26:58,158: INFO: config_utils: yaml file: config\config.yaml loaded successfully]
[2025-01-17 07:26:58,159: INFO: config_utils: yaml file: params.yaml loaded successfully]
[2025-01-17 07:26:58,160: INFO: file_utils: created directory at: artifacts]
[2025-01-17 07:26:58,163: INFO: file_utils: created directory at: artifacts/baseline_modeling]
[2025-01-17 07:26:58,164: INFO: 1409304167: Initialized TrainingManager with output directory: artifacts\baseline_modeling\20250117_072658]
[2025-01-17 07:26:58,165: INFO: 3604585105: Initialized SentimentPipeline with output directory: artifacts\baseline_modeling\20250117_072658]
[2025-01-17 07:26:58,166: INFO: 3604585105: Starting model training pipeline]
[2025-01-17 07:26:58,168: INFO: 3604585105: Loading prepared data from artifacts...]
[2025-01-17 07:26:58,193: INFO: 3604585105: Data successfully loaded and converted to sparse format]
[2025-01-17 07:26:58,398: INFO: 3604585105: Training model: logistic_regression]
[2025-01-17 07

d:\AI\NLP\HandsOn\sentiment-analysis\SentiScope-env\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


[2025-01-17 07:27:15,331: INFO: 1409304167: Making predictions for logistic_regression...]
[2025-01-17 07:27:15,340: INFO: 1409304167: Calculating metrics for logistic_regression...]
[2025-01-17 07:27:15,361: INFO: 1409304167: Successfully completed training and evaluation for logistic_regression]
[2025-01-17 07:27:15,364: INFO: 3604585105: Saved model logistic_regression to artifacts\baseline_modeling\20250117_072658\models\logistic_regression.joblib]
[2025-01-17 07:27:15,365: INFO: 3604585105: Model training pipeline completed successfully]
{'logistic_regression': TrainingResult(model_name='logistic_regression', metrics={'0': {'precision': 0.8866193144218467, 'recall': 0.9321860939746741, 'f1-score': 0.9088319088319088, 'support': 21559.0}, '1': {'precision': 0.8566114933978055, 'recall': 0.8223531512229959, 'f1-score': 0.8391328110766989, 'support': 11202.0}, '2': {'precision': 0.7506227246598965, 'recall': 0.7059830600108128, 'f1-score': 0.7276188707280832, 'support': 11098.0}, 'ac

-------

In [14]:
import xgboost as xgb
class XGBoostModel(SklearnModelWrapper):
    """XGBoost implementation using sklearn API"""
    
    def __init__(self, num_classes=3, params=None):
        default_params = {
            'objective': 'multi:softmax',
            'num_class': num_classes,
            'eta': 0.3,
            'max_depth': 6,
            'eval_metric': 'merror'

        }
        if params:
            default_params.update(params)
        super().__init__(xgb.XGBClassifier(**default_params))

In [15]:
# Define XGBoost parameters
xgb_params = {
    'objective': 'multi:softmax',
    'num_class': 3,
    'eta': 0.3,
    'max_depth': 6,
    'eval_metric': 'merror'
}

In [16]:
pipeline.models['xgboost'] = XGBoostModel(num_classes=3, params=xgb_params)
results = pipeline.train_models()
print(results)

[2025-01-17 07:31:38,373: INFO: 3604585105: Starting model training pipeline]
[2025-01-17 07:31:38,374: INFO: 3604585105: Loading prepared data from artifacts...]
[2025-01-17 07:31:38,403: INFO: 3604585105: Data successfully loaded and converted to sparse format]
[2025-01-17 07:31:38,405: INFO: 3604585105: Training model: logistic_regression]
[2025-01-17 07:31:38,406: INFO: 1409304167: Starting training and evaluation for logistic_regression]
[2025-01-17 07:31:38,406: INFO: 1409304167: Validating data split...]
[2025-01-17 07:31:38,407: INFO: 1409304167: Data split validation completed successfully]
[2025-01-17 07:31:38,407: INFO: 1409304167: Training logistic_regression...]


d:\AI\NLP\HandsOn\sentiment-analysis\SentiScope-env\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


[2025-01-17 07:31:54,800: INFO: 1409304167: Making predictions for logistic_regression...]
[2025-01-17 07:31:54,809: INFO: 1409304167: Calculating metrics for logistic_regression...]
[2025-01-17 07:31:54,829: INFO: 1409304167: Successfully completed training and evaluation for logistic_regression]
[2025-01-17 07:31:54,831: INFO: 3604585105: Saved model logistic_regression to artifacts\baseline_modeling\20250117_072658\models\logistic_regression.joblib]
[2025-01-17 07:31:54,832: INFO: 3604585105: Training model: xgboost]
[2025-01-17 07:31:54,834: INFO: 1409304167: Starting training and evaluation for xgboost]
[2025-01-17 07:31:54,835: INFO: 1409304167: Validating data split...]
[2025-01-17 07:31:54,836: INFO: 1409304167: Data split validation completed successfully]
[2025-01-17 07:31:54,836: INFO: 1409304167: Training xgboost...]
[2025-01-17 07:32:01,142: INFO: 1409304167: Making predictions for xgboost...]
[2025-01-17 07:32:01,295: INFO: 1409304167: Calculating metrics for xgboost...]
